In [2]:
!pip show pymongo

Name: pymongo
Version: 3.10.1
Summary: Python driver for MongoDB <http://www.mongodb.org>
Home-page: http://github.com/mongodb/mongo-python-driver
Author: Mike Dirolf
Author-email: mongodb-user@googlegroups.com
License: Apache License, Version 2.0
Location: c:\anaconda3\lib\site-packages
Requires: 
Required-by: 


### Pymongo
* Create Database, Collection 
* CRUD the document in Collection

In [8]:
import pymongo

## Create connection
conn = pymongo.MongoClient(host = 'localhost', port = 27017)

## Create database
db = conn.pymongo_db ## right side is the name that we give this line
# print(db.name)

## Create collection
col = db.pymongo_col ## same context with above one
# print(col.name)



#### insert_one(), insert_many()

In [32]:
post = {
    "author"  : "길동",
    "text" : "첫번째 글",
    "tags" : [
        "mongodb",
        "python",
        "pymongo"
    ]
}
result = col.insert_one(post)
print(result.inserted_id)

5f22624f6ec8d0487dd681ce


In [11]:
col.find()

In [16]:
for post in col.find():
    print('Type : ' , type(post))
    print(post)

Type :  <class 'dict'>
{'_id': ObjectId('5f225fe36ec8d0487dd681bd'), 'author': '길동', 'text': '첫번째 글', 'tags': ['mongodb', 'python', 'pymongo']}


In [17]:
##  # of document 
col.count_documents({})

1

In [33]:
post = [
    {
        "author"  : "둘리",
        "age" : 10
    },
    {
        "author"  : "바다",
        "age" : 20
    },
    {
        "author"  : "연못",
        "age" : 30
    },
]
col.insert_many(post)


In [41]:
print(col.count_documents({}))
for post in col.find({},{"author": 1, "age" :1, "text": 1, "_id" :0}).sort("age", pymongo.DESCENDING):
    print(post)

4
{'author': '연못', 'age': 30}
{'author': '바다', 'age': 20}
{'author': '둘리', 'age': 10}
{'author': '길동', 'text': '첫번째 글'}


In [43]:
# col.delete_many({"age" : 10})
# col.delete_many({})

In [49]:
emp = db.employees
print(emp)

Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'pymongo_db'), 'employees')


In [52]:
from datetime import datetime
emp_list = [
    {
        "number" : 1001,
        "last_name" : "Smith",
        "first_name" : "John",
        "salary" : 62000,
        "department" : "sales",
        "hire_date" : datetime.strptime("2016-01-02", "%Y-%m-%d")},
    {
        "number" : 1002,
        "last_name" : "Anderson",
        "first_name":"Jane",
        "salary":57500,
        "department":"marketing",
        "hire_date":datetime.strptime("2013-11-09", "%Y-%m-%d")},
    {
        "number" : 1003,
        "last_name" : "Everest",
        "first_name" : "Brad",
        "salary" : 71000,
        "department" : "sales",
        "hire_date" :datetime.strptime("2017-02-03", "%Y-%m-%d")},
    {
        "number" : 1004,
        "last_name" : "Horvath",
        "first_name" : "Jack",
        "salary" : 42000,
        "department" : "marketing",
        "hire_date" :datetime.strptime("2017-06-01", "%Y-%m-%d")},
]
emp.insert_many(emp_list)

In [59]:
## Find someone whose name have 'a'
query = {"first_name" : {"$regex" : "a"}}

## Find someone whose name start with 'J'
query = {"first_name" : {"$regex" : "^J"}}

## Find someone whose name end with 'k'
query = {"first_name" : {"$regex" : "k$"}}

for employee in emp.find(query, {"_id" : 0}):
    print(employee)


{'number': 1004, 'last_name': 'Horvath', 'first_name': 'Jack', 'salary': 42000, 'department': 'marketing', 'hire_date': datetime.datetime(2017, 6, 1, 0, 0)}


In [66]:
from datetime import datetime

from_dt = datetime.strptime("2016-01-01", '%Y-%m-%d')
to_dt = datetime.strptime("2017-05-30", '%Y-%m-%d')

query = {
    "hire_date" : {"$gte" : from_dt, "$lte" : to_dt}
}

query = {
    "number" :{"$in" : [1001,1002]}
}

for employee in emp.find(query , {"_id" : 0}):
    print(employee)

{'number': 1001, 'last_name': 'Smith', 'first_name': 'John', 'salary': 62000, 'department': 'sales', 'hire_date': datetime.datetime(2016, 1, 2, 0, 0)}
{'number': 1002, 'last_name': 'Anderson', 'first_name': 'Jane', 'salary': 57500, 'department': 'marketing', 'hire_date': datetime.datetime(2013, 11, 9, 0, 0)}


In [71]:
query = {
    "$set" : {"department" : "sales"}
}
emp.update_one({"number" : 1001},query)

query = {
    "$set" : {"commission" : 0 }
}
emp.update_many({"department" : "sales"}, query)

for employee in emp.find({"department" : "sales"}):
    print(employee)


{'_id': ObjectId('5f2266606ec8d0487dd681d6'), 'number': 1001, 'last_name': 'Smith', 'first_name': 'John', 'salary': 62000, 'department': 'sales', 'hire_date': datetime.datetime(2016, 1, 2, 0, 0), 'commission': 0}
{'_id': ObjectId('5f2266606ec8d0487dd681d8'), 'number': 1003, 'last_name': 'Everest', 'first_name': 'Brad', 'salary': 71000, 'department': 'sales', 'hire_date': datetime.datetime(2017, 2, 3, 0, 0), 'commission': 0}


In [75]:
query = {
    "$inc" : {"commission" : 100}
}
emp.update_many({"department" : "sales"}, query)
for employee in emp.find({"department" : "sales"}):
    print(employee)

{'_id': ObjectId('5f2266606ec8d0487dd681d6'), 'number': 1001, 'last_name': 'Smith', 'first_name': 'John', 'salary': 62000, 'department': 'sales', 'hire_date': datetime.datetime(2016, 1, 2, 0, 0), 'commission': 100}
{'_id': ObjectId('5f2266606ec8d0487dd681d8'), 'number': 1003, 'last_name': 'Everest', 'first_name': 'Brad', 'salary': 71000, 'department': 'sales', 'hire_date': datetime.datetime(2017, 2, 3, 0, 0), 'commission': 100}


In [91]:
def prt():
    for employee in emp.find({},{"_id" : 0}):
        print(employee)

In [92]:
emp.update_many({},{"$set" : {"status" : "A"}})
emp.update_many({"number" : {"$in" : [1003,1004] }},{"$set" : {"status" : "B"}})
emp.delete_many({"status" : "B"})
prt()

{'number': 1001, 'last_name': 'Smith', 'first_name': 'John', 'salary': 62000, 'department': 'sales', 'hire_date': datetime.datetime(2016, 1, 2, 0, 0), 'commission': 100, 'status': 'A'}
{'number': 1002, 'last_name': 'Anderson', 'first_name': 'Jane', 'salary': 57500, 'department': 'marketing', 'hire_date': datetime.datetime(2013, 11, 9, 0, 0), 'status': 'A'}
